In [ ]:
#| default_exp models.gru

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

#  GRU
> Cho et. al proposed the Gated Recurrent Unit (`GRU`) to improve on LSTM and Elman cells. The predictions at each time are given by a fully connected decoder.<br><br>**References**<br>-[Junyoung Chung, Caglar Gulcehre, KyungHyun Cho, Yoshua Bengio (2014). "Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling".](https://arxiv.org/abs/1412.3555)<br>-[Kyunghyun Cho, Bart van Merrienboer, Dzmitry Bahdanau, Yoshua Bengio (2014). "On the Properties of Neural Machine Translation: Encoder-Decoder Approaches".](https://arxiv.org/abs/1409.1259)<br>

![Figure 1. Gated Recurrent Unit Cell.](imgs_models/gru.png)

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from neuralforecast.utils import generate_series

In [ ]:
#| export
import torch
import torch.nn as nn

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_recurrent import BaseRecurrent

In [ ]:
#| export
class GRU(BaseRecurrent):
    def __init__(self,
                 input_size: int,
                 h: int,
                 state_hsize: int = 200, 
                 step_size: int = 1,
                 n_layers: int = 2,
                 activation: str = 'tanh',
                 bias: bool = True,
                 dropout: float = 0.,
                 learning_rate: float = 1e-3,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 normalize: bool = True,
                 loss=MAE(),
                 batch_size=32, 
                 num_workers_loader=0,
                 drop_last_loader=False,
                 random_seed=1,
                 **trainer_kwargs):
        super(GRU, self).__init__(
            loss=loss,
            batch_size=batch_size,
            futr_exog_list=futr_exog_list,
            hist_exog_list=hist_exog_list,
            stat_exog_list=stat_exog_list,
            num_workers_loader=num_workers_loader,
            drop_last_loader=drop_last_loader,
            random_seed=random_seed,
            **trainer_kwargs
        )

        # Architecture
        self.input_size = input_size
        self.h = h
        self.state_hsize = state_hsize
        self.step_size = step_size
        self.n_layers = n_layers
        self.activation = activation
        self.bias = bias
        self.dropout = dropout

        self.futr_exog_size = len(self.futr_exog_list)
        self.hist_exog_size = len(self.hist_exog_list)
        self.stat_exog_size = len(self.stat_exog_list)

        input_rnn = input_size + self.hist_exog_size*input_size + \
                    self.futr_exog_size*(input_size + h) + self.stat_exog_size

        # Optimization
        self.learning_rate = learning_rate
        self.loss = loss
        self.normalize = normalize
        self.random_seed = random_seed
        self.padder = nn.ConstantPad1d(padding=(0, self.h), value=0)

        # Instantiate model
        self.gru = nn.GRU(input_size=input_rnn,
                            hidden_size=self.state_hsize,
                            num_layers=self.n_layers,
                            bias=self.bias,
                            dropout=self.dropout,
                            batch_first=True)
        self.adapterW  = nn.Linear(self.state_hsize, self.h)

    def forward(self, windows_batch):
        
        # Parse windows_batch
        insample_y    = windows_batch['insample_y']
        futr_exog     = windows_batch['futr_exog']
        hist_exog     = windows_batch['hist_exog']
        stat_exog     = windows_batch['stat_exog']

        # Flatten inputs [B, W, C, L+H] -> [B, W, C*(L+H)]
        # Contatenate [ Y_t, | X_{t-L},..., X_{t} | F_{t-L},..., F_{t+H} | S ]
        batch_size, windows_size = insample_y.shape[:2]
        if self.hist_exog_size > 0:
            hist_exog = hist_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, hist_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.futr_exog_size > 0:
            futr_exog = futr_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, futr_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.stat_exog_size > 0:
            insample_y = torch.cat(( insample_y, stat_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        # GRU forward
        insample_y, _ = self.gru(insample_y)
        insample_y = self.adapterW(insample_y)
        
        return insample_y

In [ ]:
#| hide
import logging
import warnings
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [ ]:
#| hide
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import pandas as pd
from neuralforecast.utils import AirPassengersDF as Y_df
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader

# Add second series
Y_df_2 = Y_df.tail(100).copy()
Y_df_2['unique_id'] = 2.0
Y_df_2['y'] = 0.5*Y_df_2['y']
Y_df = Y_df.append(Y_df_2).reset_index(drop=True)

# Train/Test split
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test

dataset, *_ = TimeSeriesDataset.from_df(df = Y_train_df)
model = GRU(24, 12, learning_rate=1e-3, max_epochs=100)
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Y_test_df['GRU'] = y_hat

In [ ]:
#| hide
pd.concat([Y_train_df[Y_train_df['unique_id']==1.0], Y_test_df[Y_test_df['unique_id']==1.0]]).drop('unique_id', axis=1).set_index('ds').plot()
pd.concat([Y_train_df[Y_train_df['unique_id']==2.0], Y_test_df[Y_test_df['unique_id']==2.0]]).drop('unique_id', axis=1).set_index('ds').plot()